In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
from scipy.stats import spearmanr

In [5]:
pd.set_option('display.expand_frame_repr', False)
np.random.seed(42)

PROJECT_DIR = Path('..', '..')
DATA_DIR = PROJECT_DIR / 'data'

In [6]:
def get_backtest_data(predictions='lasso/predictions'):
    """Combine chapter 7 lr/lasso/ridge regression predictions
        with adjusted OHLCV Quandl Wiki data"""
    with pd.HDFStore(DATA_DIR / 'assets.h5') as store:
        prices = (store['finance_datareader/prices']
                  .filter(like='adj')
                  .rename(columns=lambda x: x.replace('adj_', ''))
                  .swaplevel(axis=0))

    with pd.HDFStore(PROJECT_DIR / '07_linear_models/data.h5') as store:
        print(store.info())
        predictions = store[predictions]

    best_alpha = predictions.groupby('alpha').apply(lambda x: spearmanr(x.actuals, x.predicted)[0]).idxmax()
    predictions = predictions[predictions.alpha == best_alpha]
    predictions.index.names = ['ticker', 'date']
    tickers = predictions.index.get_level_values('ticker').unique()
    start = predictions.index.get_level_values('date').min().strftime('%Y-%m-%d')
    stop = (predictions.index.get_level_values('date').max() + pd.DateOffset(1)).strftime('%Y-%m-%d')
    idx = pd.IndexSlice
    prices = prices.sort_index().loc[idx[tickers, start:stop], :]
    predictions = predictions.loc[predictions.alpha == best_alpha, ['predicted']]
    return predictions.join(prices, how='right')

In [7]:
df = get_backtest_data('lasso/predictions')
print(df.info())
df.to_hdf('backtest.h5', 'data')

<class 'pandas.io.pytables.HDFStore'>
File path: ..\..\07_linear_models\data.h5
/lasso/coeffs                 frame        (shape->[8,142])     
/lasso/predictions            frame        (shape->[589824,3])  
/lasso/scores                 frame        (shape->[6000,3])    
/lr/predictions               frame        (shape->[73728,2])   
/lr/scores                    frame        (shape->[750,2])     
/model_data                   frame        (shape->[799733,177])
/ridge/coeffs                 frame        (shape->[18,142])    
/ridge/predictions            frame        (shape->[1327104,3]) 
/ridge/scores                 frame        (shape->[13500,3])   
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 226941 entries, ('000100', Timestamp('2014-11-12 00:00:00')) to ('001340', Timestamp('2017-11-29 00:00:00'))
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   predicted  73728 non-null   float64
 1   close      22694